# DCGAN
*Zhiang Chen, April 2017*

Using the package: https://github.com/sugyan/tf-dcgan

### 1. Import packages

In [1]:
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
import matplotlib.pyplot as plt
import random
import operator
import time
import os
import math
import deepdish as dd
from tensorflow.contrib.learn.python.learn.datasets.mnist import read_data_sets
from math import *
import time
from dcgan import DCGAN
from datetime import datetime

### 2. Import data

In [2]:
wd = os.getcwd()
os.chdir('..')
file_name = 'resized_depth_data2.h5'

save = dd.io.load(file_name)

train_objects = save['train_objects']
train_orientations = save['train_orientations']
train_values = save['train_values']
valid_objects = save['valid_objects']
valid_orientations = save['valid_orientations']
valid_values = save['valid_values']
test_objects = save['test_objects']
test_orientations = save['test_orientations']
test_values = save['test_values']
value2object = save['value2object']
object2value = save['object2value']
del save

os.chdir(wd)

print('training dataset', train_objects.shape, train_orientations.shape, train_values.shape)
print('validation dataset', valid_objects.shape, valid_orientations.shape, valid_values.shape)
print('testing dataset', test_objects.shape, test_orientations.shape, test_values.shape)

('training dataset', (427680, 11), (427680, 10), (427680, 48, 48))
('validation dataset', (71226, 11), (71226, 10), (71226, 48, 48))
('testing dataset', (71280, 11), (71280, 10), (71280, 48, 48))


### 3. Shuffle data

In [3]:
image_size = 48

def randomize(dataset, classes, angles):
    permutation = np.random.permutation(classes.shape[0])
    shuffled_dataset = dataset[permutation,:,:]
    shuffled_classes = classes[permutation]
    shuffled_angles = angles[permutation]
    return shuffled_dataset, shuffled_classes, shuffled_angles

train_dataset, train_classes, train_angles = randomize(train_values, train_objects, train_orientations)
valid_dataset, valid_classes, valid_angles = randomize(valid_values, valid_objects, valid_orientations)
test_dataset, test_classes, test_angles = randomize(test_values, test_objects, test_orientations)

train_dataset = train_dataset[:150000,:,:]
train_angles = train_angles[:150000,:]
train_classes = train_classes[:150000,:]

valid_dataset = valid_dataset[:5000,:,:]
valid_angles = valid_angles[:5000,:]
valid_classes = valid_classes[:5000,:]

test_dataset = test_dataset[:5000,:,:]
test_angles = test_angles[:5000,:]
test_classes = test_classes[:5000,:]

train_dataset = train_dataset.reshape(-1,image_size,image_size,1)
test_dataset = test_dataset.reshape(-1,image_size,image_size,1)
n_samples = train_dataset.shape[0]

### 4. DCGAN

In [4]:
FLAGS = tf.app.flags.FLAGS

tf.app.flags.DEFINE_string('logdir', 'logdir',
                           """Directory where to write event logs and checkpoint.""")
tf.app.flags.DEFINE_integer('max_steps', 80000,
                            """Number of batches to run.""")
tf.app.flags.DEFINE_string('images_dir', 'images',
                           """Directory where to write generated images.""")

In [5]:
np.random.seed(0)
tf.set_random_seed(0)
s_size = 3 # s_size*2**4 == image_size

dcgan = DCGAN(s_size=s_size)
batch_size = dcgan.batch_size #128
min_queue_examples = 5000

train_images = tf.train.shuffle_batch([train_dataset], \
                                      batch_size=batch_size, \
                                      capacity=min_queue_examples + 3 * batch_size, \
                                      min_after_dequeue=min_queue_examples, \
                                      enqueue_many = True)

test_images = tf.train.shuffle_batch([test_dataset], \
                                     batch_size=batch_size, \
                                     capacity=min_queue_examples + 3 * batch_size, \
                                     min_after_dequeue=min_queue_examples, \
                                     enqueue_many = True)

losses = dcgan.loss(train_images)

In [6]:
# feature matching
graph = tf.get_default_graph()
features_g = tf.reduce_mean(graph.get_tensor_by_name('dg/d/conv4/outputs:0'), 0)
features_t = tf.reduce_mean(graph.get_tensor_by_name('dt/d/conv4/outputs:0'), 0)
losses[dcgan.g] += tf.multiply(tf.nn.l2_loss(features_g - features_t), 0.05)

tf.summary.scalar('g loss', losses[dcgan.g])
tf.summary.scalar('d loss', losses[dcgan.d])
train_op = dcgan.train(losses, learning_rate=0.0001)
summary_op = tf.summary.merge_all()

g_saver = tf.train.Saver(dcgan.g.variables, max_to_keep=15)
d_saver = tf.train.Saver(dcgan.d.variables, max_to_keep=15)
g_checkpoint_path = os.path.join(FLAGS.logdir, '/g.ckpt')
d_checkpoint_path = os.path.join(FLAGS.logdir, '/d.ckpt')

config = tf.ConfigProto()
config.gpu_options.allow_growth=True
config.log_device_placement = True
config.gpu_options.allocator_type = 'BFC' 

INFO:tensorflow:Summary name g loss is illegal; using g_loss instead.
INFO:tensorflow:Summary name d loss is illegal; using d_loss instead.


In [7]:
with tf.Session(config=config) as sess:
    summary_writer = tf.summary.FileWriter(FLAGS.logdir, graph=sess.graph)
    # restore or initialize generator
    sess.run(tf.global_variables_initializer())
    '''
    if os.path.exists(g_checkpoint_path):
        print('restore variables:')
        for v in dcgan.g.variables:
            print('  ' + v.name)
        g_saver.restore(sess, g_checkpoint_path)
    if os.path.exists(d_checkpoint_path):
        print('restore variables:')
        for v in dcgan.d.variables:
            print('  ' + v.name)
        d_saver.restore(sess, d_checkpoint_path)
    '''
        
    try: 
        g_saver.restore(sess, './logdir/g.ckpt')
        print ('restore variables:')
        for v in dcgan.g.variables:
            print('  ' + v.name)
    except:
        print "g using random initialization"
            
    try: 
        d_saver.restore(sess, './logdir/d.ckpt')
        print('restore variables:')
        for v in dcgan.d.variables:
            print('  ' + v.name)
    except:
        print "d using random initialization"
        
    # setup for monitoring
    sample_z = sess.run(tf.random_uniform([dcgan.batch_size, dcgan.z_dim], minval=-1.0, maxval=1.0))
    images = dcgan.sample_images(5, 5, inputs=sample_z)

    # start training
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    
    dcgan.retrain
    
    for step in range(FLAGS.max_steps):
        start_time = time.time()
        _, g_loss, d_loss = sess.run([train_op, losses[dcgan.g], losses[dcgan.d]])
        duration = time.time() - start_time
        
        if step%20 == 0:
            print('{}: step {:5d}, loss = (G: {:.8f}, D: {:.8f}) ({:.3f} sec/batch)'.format(
                datetime.now(), step, g_loss, d_loss, duration))

        # save generated images
        if step % 100 == 0:
            # summary
            summary_str = sess.run(summary_op)
            summary_writer.add_summary(summary_str, step)
            # sample images
            filename = os.path.join(FLAGS.images_dir, '%05d.jpg' % step)
            with open(filename, 'wb') as f:
                f.write(sess.run(images))
        # save variables
        if (step+1) == FLAGS.max_steps:
            g_saver.save(sess, './logdir/g.ckpt')
            d_saver.save(sess, './logdir/d.ckpt')
            
    coord.request_stop()
    coord.join(threads)

g using random initialization
d using random initialization
INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.InternalError'>, Dst tensor is not initialized.
	 [[Node: shuffle_batch/Const = Const[dtype=DT_FLOAT, value=Tensor<type: float shape: [150000,48,48,1] values: [[[-0.0024799407][-0.047286831][-0.062222458]]]...>, _device="/job:localhost/replica:0/task:0/gpu:0"]()]]

Caused by op u'shuffle_batch/Const', defined at:
  File "/usr/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/zhiang/.local/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/zhiang/.local/lib/python2.7/site-packages/traitlets/config/application.py", line 653, in launch_instance
    app.start()
  File "/home/zhiang/.local/lib/python2.7/site-packages/ipykerne

OutOfRangeError: RandomShuffleQueue '_1_shuffle_batch/random_shuffle_queue' is closed and has insufficient elements (requested 128, current size 0)
	 [[Node: shuffle_batch = QueueDequeueManyV2[component_types=[DT_FLOAT], timeout_ms=-1, _device="/job:localhost/replica:0/task:0/cpu:0"](shuffle_batch/random_shuffle_queue, shuffle_batch/n)]]
	 [[Node: shuffle_batch/_163 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/gpu:0", send_device="/job:localhost/replica:0/task:0/cpu:0", send_device_incarnation=1, tensor_name="edge_1237_shuffle_batch", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/gpu:0"]()]]

Caused by op u'shuffle_batch', defined at:
  File "/usr/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/zhiang/.local/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    ap